# 一、python环境

## Anaconda
[Anaconda](https://repo.anaconda.com/archive/)版本大全
```bash
wget https://repo.anaconda.com/archive/Anaconda3-2022.05-Linux-x86_64.sh
bash Anaconda3-2022.05-Linux-x86_64.sh
```

## pytorch
[pytorch](https://pytorch.org/get-started/previous-versions/)官网查询安装

### 1、RuntimeError: DataLoader worker (pid 25846) is killed by signal: Killed.
问题原因：Dataloader的num_workers过大，导致内存不足
解决办法：
```bash
# 1、增加容器内存(没尝试过)
--shm-size 64G
# 2、减少num_workers
# 3、增加虚拟内存(没啥用)
cd /root
fallocate -l 4G /swapfile                                   #设置4G交换内存
ls -lh /swapfile                                            #查看是否创建成功
chmod 600 /swapfile                                         #激活虚拟内存
mkswap /swapfile
swapon /swapfile
swapon --show                                               #查看
cp /etc/fstab /etc/fstab.bak                                #备份配置文件
echo '/swapfile none swap sw 0 0' | sudo tee -a /etc/fstab  #将虚拟内存固化
free -mh                                                    #查看结果
```

## opencv

### 1、ImportError: libGL.so.1: cannot open shared object file: No such file or directory
原因：缺少依赖库  
解决办法：
```bash
#1、安装完整opencv
pip install opencv-python-headless
#2、安装缺少的库
apt install libgl1-mesa-glx
```

### 2、cv2.error: OpenCV(4.6.0) /io/opencv/modules/highgui/src/window.cpp:1267: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
解决办法：
```bash
apt-get update
apt-get install libgtk2.0-dev pkg-config
```

## apex

### 1、ModuleNotFoundError: No module named 'apex'
[apex](https://github.com/NVIDIA/apex)官网下载安装包  
执行
```bash
unzip apex-master.zip
cd apex-master
pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
```

## pycocotools

问题：无法安装  
解决办法：
```bash
conda install -c conda-forge pycocotools
```

## mmcv

### 1、ModuleNotFoundError: No module named 'mmcv._ext'
原因：mmcv与cuda或pytorch版本不对应
解决办法：寻找对应的版本
```bash
 #打印所有mmcv版本
 pip install mmcv-full==latest+torch1.12.0+cu110 -f https://download.openmmlab.com/mmcv/dist/index.html
 #安装对应版本
 pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu110/torch1.7.0/index.html
```

## 国内pip源
### 1、豆瓣源
```bash
pip install -i https://pypi.doubanio.com/simple {package}
```
### 2、清华源
```bash
pip install -i https://pypi.tuna.tsinghua.edu.cn/simple {package}
```

# 二、ubuntu

## 查询CPU
***
```bash
cat /proc/cpuinfo | grep name | cut -f2 -d: | uniq -c
```

## 禁止ubuntu内核更新：
***
```bash
uname -a   #查看正在使用的内核版本
dpkg --get-selections |grep linux-image    #查看安装好的内核版本
sudo apt-mark hold linux-image-x.xx.x-xx-generic
sudo apt-mark hold linux-image-extra-x.xx.x-xx-generic
sudo apt-mark hold nvidia-415
```

## linux安装SSH+允许root用户远程登录，并允许X11
***
```bash
apt-get update
apt-get install openssh-server
/etc/init.d/ssh start
apt-get install vim
vim /etc/ssh/sshd_config
PermitRootLogin prohibit-password --> PermitRootLogin yes
LoginGraceTime 2m --> LoginGraceTime 120
AddressFamily any --> AddressFamily inet
passwd root #设置root用户
/etc/init.d/ssh restart
```

## 导入tar镜像文件
***
```bash
sudo dockerd -s overlay2
nvidia-docker load --input ./chipeak-ub18.04-cuda10.1-detectron2-zwh-2020-06-16.tar
```

## 导出镜像为tar文件
***
```bash
nvidia-docker save -o chipeak-ub18.04-cuda10.1-detectron2-qhy-2021-03-25.tar  [镜像id或name]
```

## 导出容器为镜像：
***
```bash
nvidia-docker commit [选项] [容器ID或容器名]  [仓库名:标签]
nvidia-docker commit -a 'weikaixxxxxx' -m '导入了数据库' 9adeb5943045  mysql:v2
# -a:修改人
# -m:备注
```

## 创建容器
***
```bash
nvidia-docker run --runtime=nvidia --gpus all   -dit --restart=always  --privileged   -v /tmp/.X11-unix:/tmp/.X11-unix   -v /work/disk2/my_work/6.qhy/Reid-2021-03-25:/work    -p 10087:22    -e DISPLAY=:0    -e LANG=C.UTF-8  --shm-size 32G   --name  ub18.04-cuda10.1-server-qhy   -w /work   329ed837d508 /bin/bash

nvidia-docker run
-dit                                       # 后台运行容器
--restart=always                           # docker重启时,自动重启
--net=bredge-zwh                           # 网桥名称
-p 10086:22                                # 端口映射
-v   /work:/my_work                        # 目录映射
-v /tmp/.X11-unix:/tmp/.X11-unix           # 图形界面信息窗体sockt
--privileged                               # container内的root拥有真正的root权限
-e DISPLAY=:0                              # 图形界面端口
-e LANG=C.UTF-8                            # 支持中文环境
--shm-size 16G                             # 共享内存
--name  chipeak-ub18.04-cuda10.1-caffe-zwh # 容器名称
9e47e9dfcb9a                               # 进行id
/bin/bash                                  # 启动终端
QT_X11_NO_MITSHM=1
-w /home/jello                             # 指定工作目录
--restart=always                           # docker重启时,自动重启
-e LANG=C.UTF-8                            # 支持中文环境
-p                                         # 本机端口——>容器端口映射
-v                                         # 挂在宿主文件，到容器。
-name                                      # 容器名称
-c                                         # 接下来的表示一个字符串
```

## docker界面显示
***
```bash
echo $DISPLAY
sudo apt-get install lightdm
sudo service lightdm start
xhost +$HOSTNAME
export DISPLAY=:0.0
```

## 查询已经被占用的ip
***
```bash
nbtscan 192.168.1.1-254
```

## sshfs挂载
***
```bash
sudo apt-get install sshfs
sshfs ${username}@${ip}:${remoto_path} ${local_path}
```

## 拉取CUDA+CUDNN容器
******
Nvidia官方[docker](https://hub.docker.com/r/nvidia/cuda/tags?page=1)  

注意：  
base版本：  
该版本是从cuda9.0开始，包含了部署预构建cuda应用程序的最低限度（libcudart）。  
如果用户需要自己安装自己需要的cuda包，可以选择使用这个image版本，但如果想省事儿，则不建议使用该image，会多出许多麻烦。  
runtime版本：  
该版本通过添加cuda工具包中的所有共享库开扩展基本image。如果使用多个cuda库的预构建应用程序，可使用此image。但是如果想借助cuda中的头文件对自己的工程进行编译，则会出现找不到文件的错误。  
devel版本：  
通过添加编译器工具链，测试工具，头文件和静态库来扩展运行的image，使用此图像可以从源代码编译cuda应用程序。

## 安装Nvidia-docker
```bash
# step 1: 安装必要的一些系统工具
sudo apt-get update
sudo apt-get -y install apt-transport-https ca-certificates curl software-properties-common
# step 2: 安装GPG证书
curl -fsSL http://mirrors.aliyun.com/docker-ce/linux/ubuntu/gpg | sudo apt-key add -
# Step 3: 写入软件源信息
sudo add-apt-repository "deb [arch=amd64] http://mirrors.aliyun.com/docker-ce/linux/ubuntu $(lsb_release -cs) stable"
# Step 4: 更新并安装 Docker-CE
sudo apt-get -y update
sudo apt-get -y install docker-ce

# 清理以前的。If you have nvidia-docker 1.0 installed: we need to remove it and all existing GPU containers
sudo docker volume ls -q -f driver=nvidia-docker | xargs -r -I{} -n1 docker ps -q -a -f volume={} | xargs -r docker rm -f
sudo apt-get purge -y nvidia-docker
sudo apt autoremove
 
# 执行命令。Add the package repositories
# command 1
curl -s -L https://nvidia.github.io/nvidia-docker/gpgkey | \
  sudo apt-key add -
 
# command 2
distribution=$(. /etc/os-release;echo $ID$VERSION_ID)
 
# command 3
curl -s -L https://nvidia.github.io/nvidia-docker/$distribution/nvidia-docker.list | \
  sudo tee /etc/apt/sources.list.d/nvidia-docker.list
sudo apt-get update
 
# 正式安装。Install nvidia-docker2 and reload the Docker daemon configuration
sudo apt-get install -y nvidia-docker2
sudo pkill -SIGHUP dockerd
 
# 测试一下。 Test nvidia-smi with the latest official CUDA image
sudo docker run --runtime=nvidia --rm nvidia/cuda:11.1.1-cudnn8-devel-ubuntu18.04  nvidia-smi
sudo groupadd docker
sudo usermod  -aG docker ${用户名}
sudo service docker restart
```
报错： ‘gpg: 找不到有效的 OpenPGP‘ & ‘E: 无法定位软件包 nvidia-docker2‘
```bash
# 打开ipaddress，搜索查找nvidia.github.io，即可得到对应的ip地址：

# nvidia.github.io
185.199.108.153 nvidia.github.io
185.199.109.153 nvidia.github.io
185.199.110.153 nvidia.github.io
185.199.111.153 nvidia.github.io

# 然后将上面的信息写入host文件即可：可在命令行输入sudo vi /etc/hosts，然后将上面的内容复制在文件
# 下方，保存退出即可。
# 这时候重新打开终端，再按安装指南一步一步操作的时候，就不会出现上面的问题了。
```

# 三、git

```bash
# 1、初始化git
git init
# 2、全部加入仓库
git add --all
git add .
# 3、生成ssh密钥连接GitHub
ssh-keygen -t rsa -C "799867114@qq.com"
cat ~/.ssh/id_rsa.pub
# 4、添加注释
git commit -am '第一次版本提交'
# 5、关联github仓库
git remote add origin https://github.com/passinme/notebook.git
# 6、查看是否成功关联
git remote
# 7、推送代码至github
git push -u origin master
```